### Calculating Observables for the Equilibrium case


In [1]:
import numpy as np 
import math 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
## class definition to measure equilibrium observable values
class EqObservables:
    def __init__(self,L,file_path):
        '''
        This function will initialize the instance of the class
        parameters:
            self(object): 
            L(integer): The system size
            file_path(string): location where the files are stored
        '''